In [147]:
import os
import json
import datetime
import requests
import pandas as pd
import numpy as np
import util.data_puller as api
import plotly.figure_factory as ff
import tensorflow as tf
from json import JSONEncoder

apikey = os.environ['API_KEY']
base_url = base_url = os.environ['API_BASE_URL'] + '/' + os.environ['API_CHANNEL'] + '/' + os.environ['API_F']

history_size = 60
start = str(datetime.datetime.now()).split('.')[0]
end = str(datetime.datetime(2020, 3, 1))

datetime_to = datetime.datetime.strptime(f"{datetime.datetime.now().date()} 23:59:59", "%Y-%m-%d %H:%M:%S")

# NOTE: Take the day before to ensure only completed days
datetime_to = datetime_to - datetime.timedelta(1)

# NOTE: Add 1 second to get the start of the next day so it matches history_size.
datetime_from = (datetime_to - datetime.timedelta(history_size)) + datetime.timedelta(seconds=1)

print(start, end)

2020-05-19 21:49:20 2020-03-01 00:00:00


In [148]:
df = api.apicallv3(base_url, apikey, str(start), str(end), True, False)

2020-05-19 21:49:21,003: INFO: data-puller: Requesting data from API...
2020-05-19 21:49:21,004: DEBUG: data-puller: Requesting: https://api.trendlog.io/V1/channels/20/feeds2?apikey=IE1YUFNTMJBU&start=19-05-2020 21:49:20&end=01-03-2020 00:00:00&dst=true&viewid=670&status=true&wherevalue=>0
2020-05-19 21:49:21,237: DEBUG: data-puller: Times down API call got: 583 points, call took: 0.229688 seconds
2020-05-19 21:49:21,237: INFO: data-puller: Requesting data from API...
2020-05-19 21:49:21,237: DEBUG: data-puller: Requesting: https://api.trendlog.io/V1/channels/20/feeds2?apikey=IE1YUFNTMJBU&start=19-05-2020 21:49:20&end=01-03-2020 00:00:00&dst=true&viewid=694&status=true&wherevalue=>0
2020-05-19 21:49:21,616: DEBUG: data-puller: Production amount API call got: 7450 points, call took: 0.328882 seconds
2020-05-19 21:49:21,617: INFO: data-puller: Starting datahandling...
2020-05-19 21:49:21,636: INFO: data-puller: Normalizing data...
2020-05-19 21:49:21,664: INFO: data-puller: Correcting da

In [149]:
corrs = df.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True
)

figure.show()

In [150]:
df

,1,2,3,4
0,0.381576,0.500000,0.475962,1.0
1,0.386257,0.857143,0.498035,1.0
2,0.468452,0.714286,0.455290,1.0
3,0.502902,0.500000,0.437916,1.0
4,0.198090,0.500000,0.583723,1.0
...,...,...,...,...
75,0.604194,0.428571,1.000000,0.0
76,0.889721,0.857143,0.871488,0.0
77,0.929788,0.642857,0.847263,0.0
78,0.693690,0.500000,0.991471,0.0


In [151]:
df.describe()

,1,2,3,4
count,80.000000,80.000000,80.000000,80.000000
mean,0.401879,0.520536,0.631775,0.437500
std,0.199882,0.194935,0.208314,0.499208
min,0.000000,0.000000,0.000000,0.000000
25%,0.282016,0.428571,0.470330,0.000000
50%,0.375398,0.500000,0.687921,0.000000
75%,0.508332,0.642857,0.801151,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [152]:
dataset = tf.data.Dataset.from_tensor_slices(df.values)
dataset = dataset.map(lambda row: tf.cast(row, 'float32'))
dataset = dataset.window(history_size, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(history_size))
dataset = dataset.map(lambda window: (window[:,1:], window[-1,0]))

arr = list(dataset.as_numpy_iterator())
print(arr[0])

(array([[0.5       , 0.475962  , 1.        ],
       [0.85714287, 0.4980355 , 1.        ],
       [0.71428573, 0.45529032, 1.        ],
       [0.5       , 0.43791586, 1.        ],
       [0.5       , 0.58372325, 1.        ],
       [0.35714287, 0.51955616, 1.        ],
       [0.85714287, 0.37027386, 1.        ],
       [0.42857143, 0.49398804, 1.        ],
       [0.78571427, 0.3215266 , 1.        ],
       [0.35714287, 0.49914116, 1.        ],
       [0.5       , 0.4294853 , 0.        ],
       [0.5714286 , 0.460246  , 1.        ],
       [0.42857143, 0.59708977, 0.        ],
       [0.71428573, 0.53833246, 0.        ],
       [0.5714286 , 0.4140457 , 0.        ],
       [0.71428573, 0.5320737 , 1.        ],
       [0.64285713, 0.41726393, 1.        ],
       [0.85714287, 0.4092677 , 1.        ],
       [0.2857143 , 0.60271674, 0.        ],
       [0.5714286 , 0.43236786, 0.        ],
       [0.5       , 0.54145193, 0.        ],
       [0.5       , 0.47811407, 0.        ],
       [0

In [153]:
len(arr[0])

2

In [178]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

tcx = [[]]
tcx[0] = df.values

#print(tcx)

data = json.dumps({"signature_name":"serving_default", "instances":tcx}, cls=NumpyArrayEncoder)
#print(f"Data: {data}")
headers = {"content-type":"application/json"}

json_response = requests.post("http://172.22.0.2:8501/v1/models/predictor:predict", data=data, headers=headers)
print(f"Json text: {json_response.text}")
predictions = json.loads(json_response.text)['predictions']

single_prediction = round(float(predictions[0][0]))

date = datetime.datetime.now() + datetime.timedelta(single_prediction)
date = date.date()

print(date)

print(single_prediction)

Json text: {
    "predictions": [[0.981895328]
    ]
}
2020-05-20
1
